## Spatial cross-walk

Objective steps:
This notebook is intended to relate the Housing and Urban Development (HUD) Zip code Crosswalk to a centroid County. We do this by stepping through spatial selection in 3 ways. 

First, if the ZCTA geometry is entirely encompassed by the county, then the ZCTA-County membership is 1-to-1 by 'geometry within' the County. 

Second, if the ZCTA geometry were found to overlap with multiple Counties, but its centroid 'intersects within' a single County geometry, then the ZCTA-County membership is 1-to-1 because the ZCTA 'centroid intersects' the County geometry. 

Third, if the ZCTA have not been found to have a 1-to-1


Spatial units: 1) Census Zip Code Tabulation Areas (ZCTA) and 2) Census County

Reference cross-walk: 1) Housing and Urban Development Zip-Code to County Crosswalk



The file 


Source datasets can be found at the following links. Downloading these source files can take a while, so do this step ahead of spatial analyses:
    
    US Census 2019 ZCTA Shapefiles (tl_2019_us_zcta510.shp)
    1) https://www2.census.gov/geo/tiger/TIGER2019/ZCTA5/
    
    US Census 2019 County Shapefiles (tl_2019_us_county.shp)
    2) https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/
    
    US Housing and Urban development crosswalk (ZIP-COUNTY, 4th QUARTER 2020)
    3) https://www.huduser.gov/portal/datasets/usps_crosswalk.html

In [1]:
# initialize libraries
import os, pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# Generate output file directory
if not os.path.exists(os.path.join(os.getcwd(), 'Census_2010_ZCTA_file')):
    os.mkdir(os.path.join(os.getcwd(), 'Census_2010_ZCTA_file'))
    
if not os.path.exists(os.path.join(os.getcwd(), 'Census_2010_COUNTY_file')):
    os.mkdir(os.path.join(os.getcwd(), 'Census_2010_COUNTY_file'))

In [97]:
%%time
# read in the HUD ZIP to COUNTY cross-walk spreadsheet
hud_file = pd.read_excel('ZIP_COUNTY_122020.xlsx', dtype={'ZIP':str,'COUNTY':str})
hud_file

CPU times: user 6.7 s, sys: 893 ms, total: 7.6 s
Wall time: 9.33 s


,ZIP,COUNTY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,00501,36103,0.000000,1.000000,0.000000,1.000000
1,00601,72001,0.839799,0.800983,0.871166,0.838115
2,00601,72113,0.160201,0.199017,0.128834,0.161885
3,00602,72003,1.000000,0.998801,1.000000,0.999919
4,00602,72005,0.000000,0.001199,0.000000,0.000081
...,...,...,...,...,...,...
54189,99925,02198,0.000000,0.000000,1.000000,1.000000
54190,99926,02198,0.000000,0.000000,1.000000,1.000000
54191,99927,02198,0.000000,0.000000,1.000000,1.000000
54192,99928,02130,0.000000,0.000000,1.000000,1.000000


In [98]:
hud_file.dtypes

ZIP           object
COUNTY        object
RES_RATIO    float64
BUS_RATIO    float64
OTH_RATIO    float64
TOT_RATIO    float64
dtype: object

In [99]:
# number of zip codes with multiple COUNTIES
zcta_multi = hud_file.ZIP.value_counts()
hud_multi = hud_file.loc[hud_file.ZIP.isin(zcta_multi[zcta_multi>1].index),:]

print('Number of ZIP codes with multiple COUNTY overlaps: {0}'.format(hud_multi.ZIP.nunique()))
hud_multi

Number of ZIP codes with multiple COUNTY overlaps: 11137


,ZIP,COUNTY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
1,00601,72001,0.839799,0.800983,0.871166,0.838115
2,00601,72113,0.160201,0.199017,0.128834,0.161885
3,00602,72003,1.000000,0.998801,1.000000,0.999919
4,00602,72005,0.000000,0.001199,0.000000,0.000081
5,00603,72071,0.000279,0.000000,0.000000,0.000255
...,...,...,...,...,...,...
53926,99371,53021,0.018692,0.000000,0.000000,0.015094
53927,99371,53001,0.981308,1.000000,1.000000,0.984906
53930,99403,53075,0.000101,0.000000,0.000000,0.000091
53931,99403,53003,0.998278,1.000000,1.000000,0.998453


## Read in ZCTA and COUNTY shapefiles

In [100]:
# Census 2010 ZCTA file
zcta = gpd.read_file(os.path.join(os.getcwd(),'../Downloads/tl_2019_us_zcta510/tl_2019_us_zcta510.shp'))

print(zcta.shape)
zcta.head()

(33144, 10)


,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,43451,43451,B5,G6350,S,63484186,157689,+41.3183010,-083.6174935,"POLYGON ((-83.708733 41.327326, -83.708147 41...."
1,43452,43452,B5,G6350,S,121522304,13721730,+41.5157923,-082.9809454,"POLYGON ((-83.086978 41.537796, -83.0825629999..."
2,43456,43456,B5,G6350,S,9320975,1003775,+41.6318300,-082.8393923,"(POLYGON ((-82.835577 41.710823, -82.83515 41...."
3,43457,43457,B5,G6350,S,48004681,0,+41.2673301,-083.4274872,"POLYGON ((-83.49650299999999 41.253708, -83.48..."
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((-83.222292 41.531025, -83.2222819999..."


In [101]:
%%time
# Census 2010 County file
county = gpd.read_file('/Users/jphuong/Downloads/tl_2019_us_county/tl_2019_us_county.shp')
county['COUNTYID'] = county.apply(lambda x: '{0}{1}'.format(x['STATEFP'],x['COUNTYFP']), axis=1)

print(county.shape)
county.head()

(3233, 19)
CPU times: user 3.55 s, sys: 339 ms, total: 3.89 s
Wall time: 4.4 s


,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,COUNTYID
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477652222,10690952,+41.9158651,-096.7885168,"POLYGON ((-97.019516 42.004097, -97.019519 42....",31039
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680962890,61582307,+46.2946377,-123.4244583,"POLYGON ((-123.436394 46.238197, -123.447592 4...",53069
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016819475,29089486,+34.3592729,-104.3686961,"POLYGON ((-104.567387 33.997566, -104.567717 3...",35011
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169270569,22849484,+40.7835474,-096.6886584,"POLYGON ((-96.91075099999999 40.784944, -96.91...",31109
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645188,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27366699999999 40.089399, -98.27...",31129


In [102]:
# subset to the ZCTA with multiple Counties (hud_multi)
zcta1 = zcta.loc[zcta.ZCTA5CE10.isin(hud_multi.ZIP.unique())]
zcta1

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
3,43457,43457,B5,G6350,S,48004681,0,+41.2673301,-083.4274872,"POLYGON ((-83.49650299999999 41.253708, -83.48..."
8,43464,43464,B5,G6350,S,86139720,12290475,+41.4048795,-082.9241092,"POLYGON ((-83.008331 41.415395, -83.0081189999..."
15,97826,97826,B5,G6350,S,822671852,57985,+45.6642964,-119.2323162,"POLYGON ((-119.460865 45.610154, -119.460831 4..."
18,97830,97830,B5,G6350,S,1553156453,588851,+44.9666148,-120.1831238,"POLYGON ((-120.514214 45.094893, -120.514198 4..."
24,97838,97838,B5,G6350,S,425979705,28240865,+45.8530040,-119.2872655,"POLYGON ((-119.624803 45.833443, -119.624033 4..."
...,...,...,...,...,...,...,...,...,...,...
33126,31778,31778,B5,G6350,S,274561310,2661782,+30.9749924,-083.7000748,"POLYGON ((-83.827511 30.933662, -83.827473 30...."
33127,31779,31779,B5,G6350,S,365776352,2782936,+31.1080217,-084.2111978,"(POLYGON ((-84.229152 31.00151, -84.2288169999..."
33128,31780,31780,B5,G6350,S,258455325,1926751,+32.0318703,-084.3888115,"POLYGON ((-84.497756 32.058782, -84.497715 32...."
33130,31783,31783,B5,G6350,S,256929587,4240140,+31.7589763,-083.4701253,"POLYGON ((-83.588437 31.792987, -83.5865319999..."


In [266]:
# verify that each ZIP code is represented with a unique geometry/polygon (no repeat ZCTA polygons)
zcta1.ZCTA5CE10.value_counts().sort_values(ascending=False)

23954    1
46403    1
24401    1
83869    1
68326    1
        ..
64625    1
38963    1
37387    1
38564    1
50459    1
Name: ZCTA5CE10, Length: 11103, dtype: int64

In [267]:
# test print for the zcta1 polygon centroids
zcta1.centroid

3         POINT (-83.4274838080112 41.26732955067876)
8         POINT (-82.9327146120978 41.40723107656639)
15       POINT (-119.2265844391391 45.65450943861963)
18        POINT (-120.189422940878 44.95014239701111)
24       POINT (-119.2869118241813 45.85297039860055)
                             ...                     
33126     POINT (-83.70905587515165 30.9749306455378)
33127    POINT (-84.19314264168615 31.10134525102244)
33128    POINT (-84.39094049606636 32.03104270295325)
33130    POINT (-83.47119708766552 31.76220931121244)
33131    POINT (-111.1447593840219 40.64134916236029)
Length: 11103, dtype: object

## Adapt the spatial crossmap 
#### Python function: crossmap_BG_to_ZCTA
#### Generate for US ZCTA that overlap with multiple COUNTIES

In [106]:
"""
Set parameters for iterative spatial intersections/joins
"""
# The first spatial unit
# The primary key for the first spatial unit
poly_1=zcta.loc[:,['GEOID10','ZCTA5CE10','geometry']]
uid1='ZCTA5CE10'
uid1_unit = 'ZCTA'


# The second spatial unit
# The primary key for the second spatial unit
poly_2=county.loc[:,['GEOID','COUNTYID','geometry']]
uid2='GEOID'
uid2_unit = 'COUNTY'

# generate output path
outshapefilepath=os.path.join(os.getcwd(),'1-1-ZCTA_COUNTY_crossmap', 'tl_2019_ZCTA_COUNTY.shp')

In [107]:
%%time
"""
1) identify the poly_1 with geometry that fit within poly_2 by polygon geometry
"""
# generate polygon centroid for each polygon
poly_1['centroid'] = poly_1.centroid

poly_1_within = gpd.sjoin(poly_1.set_geometry('geometry'),
                          poly_2,
                          how='inner',
                          op='within').drop('index_right', axis=1)
poly_1_within = poly_1_within.assign(overlay='geometry within {0}'.format(uid2_unit))

print(poly_1_within.shape)

(22865, 7)
CPU times: user 2min 6s, sys: 7.57 s, total: 2min 13s
Wall time: 2min 23s


In [108]:
%%time
"""
2) identify the remaining poly_1 that intersects poly_2 by polygon centroid
"""
# extract polygons that could not be joined by 'geometry within' method
poly_1_remaining1 = poly_1[~poly_1[uid1].isin(poly_1_within[uid1])]

poly_1_intersects = gpd.sjoin(poly_1_remaining1.set_geometry('centroid'),
                              poly_2,
                              how='inner',
                              op='intersects').drop('index_right', axis=1)\
    .assign(overlay='centroid intersects {0}'.format(uid2_unit))

print(poly_1_intersects.shape)

(10276, 7)
CPU times: user 2.25 s, sys: 276 ms, total: 2.52 s
Wall time: 2.71 s


In [109]:
%%time
"""
3) identify the poly_1 that intersect with one poly_2 by polygon geometry
"""
# extract polygons that could not be joined by 'centroid intersection' method
poly_1_remaining2 = poly_1_remaining1.loc[~poly_1_remaining1[uid1].isin(poly_1_intersects[uid1]),:]

poly_1_g_left = gpd.sjoin(poly_1_remaining2.set_geometry('geometry'),
                          poly_2, 
                          how='inner',
                          op='intersects').drop('index_right', axis=1)

mappings = poly_1_g_left.groupby(uid1)[uid1].count()
poly_1_g_left = poly_1_g_left.loc[poly_1_g_left[uid1].isin(mappings[mappings==1].dropna().index),:]\
    .assign(overlay='geometry intersects a single {0}'.format(uid2_unit))

print(poly_1_g_left.shape)

(0, 7)
CPU times: user 294 ms, sys: 16 ms, total: 310 ms
Wall time: 337 ms


In [110]:
"""
4) identify the poly_1 with multiple poly_2 by geometry intersection, but 0 poly_1 by centroid intersection
"""
poly_1_remaining3 = poly_1_remaining2.loc[~poly_1_remaining2[uid1].isin(poly_1_g_left[uid1]),:]

poly_1_c_left = gpd.sjoin(poly_1_remaining3.set_geometry('centroid'),
                      poly_2,
                      how='left',
                      op='intersects').drop('index_right', axis=1)
poly_1_c_left = poly_1_c_left[poly_1_c_left[uid1].isin(mappings[mappings>1].dropna().index)]\
    .assign(overlay='geometry intersects multiple {0}, centroid intersects 0 {0}'.format(uid2_unit))

print(poly_1_c_left.shape)

(3, 7)


In [113]:
"""
5) merge the spatial joins, report, then export
"""
poly_1_mapped = pd.concat([poly_1_within, poly_1_intersects, poly_1_g_left, poly_1_c_left], axis=0)\
    .rename(columns={uid2:'COUNTYFIPS'})

# coverage
print('geometry within: {0} ({1}%)'\
      .format(poly_1_within.shape[0], round(poly_1_within.shape[0]/poly_1.shape[0]*100,5)))

print('centroid intersects: {0} ({1}%)'\
      .format(poly_1_intersects.shape[0], round(poly_1_intersects.shape[0]/poly_1.shape[0]*100, 5)))

print('geometry intersects 1 County: {0} ({1}%)'\
      .format(poly_1_g_left.shape[0], round(poly_1_g_left.shape[0]/poly_1.shape[0]*100, 5)))

print('geometry intersects multiple ZCTA, centroid intersects 0 County: {0} ({1}%)'\
      .format(poly_1_c_left.shape[0], round(poly_1_c_left.shape[0]/poly_1.shape[0]*100, 5)))

print('total coverage: {0}/{1}'\
      .format(poly_1_mapped[uid1].isin(poly_1[uid1]).sum(), poly_1[uid1].nunique()))

geometry within: 22865 (68.98685%)
centroid intersects: 10276 (31.0041%)
geometry intersects 1 County: 0 (0.0%)
geometry intersects multiple ZCTA, centroid intersects 0 County: 3 (0.00905%)
total coverage: 33144/33144


In [228]:
# Merge in the COUNTY mappings based on spatial intersection
hud_file1 = hud_file.merge(poly_1_mapped.loc[:,['ZCTA5CE10','COUNTYFIPS','overlay']]\
                               .rename(columns={'COUNTYFIPS':'Spatial_Overlap_COUNTY'}),
                           how='left',
                           left_on=['ZIP'],
                           right_on=['ZCTA5CE10'])\
    .drop('ZCTA5CE10', axis=1)

# Merge in the COUNTY mappings based on majority residence e.g., RES_RATIO
hud_maxRes = hud_file1.loc[hud_file1.groupby('ZIP')['RES_RATIO'].agg('idxmax'),['ZIP','COUNTY']]\
    .rename(columns={'COUNTY':'MaxResCounty'})
hud_file1 = hud_file1.merge(hud_maxRes, on='ZIP', how='left')

# fill in the missing overlay instances
hud_file1.loc[hud_file1.overlay.isna(),'overlay'] = 'single ZIP to COUNTY association'

# assign the preferred county
hud_file1['Preferred_county'] = hud_file1.apply(lambda x: x['Spatial_Overlap_COUNTY'] 
                                                if x['overlay']=='geometry within COUNTY' else x['MaxResCounty'],
                                                axis=1)

# test print
hud_file1

,ZIP,COUNTY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,Spatial_Overlap_COUNTY,overlay,MaxResCounty,Preferred_county
0,00501,36103,0.000000,1.000000,0.000000,1.000000,NaN,single ZIP to COUNTY association,36103,36103
1,00601,72001,0.839799,0.800983,0.871166,0.838115,72001,centroid intersects COUNTY,72001,72001
2,00601,72113,0.160201,0.199017,0.128834,0.161885,72001,centroid intersects COUNTY,72001,72001
3,00602,72003,1.000000,0.998801,1.000000,0.999919,72003,geometry within COUNTY,72003,72003
4,00602,72005,0.000000,0.001199,0.000000,0.000081,72003,geometry within COUNTY,72003,72003
...,...,...,...,...,...,...,...,...,...,...
54189,99925,02198,0.000000,0.000000,1.000000,1.000000,02198,geometry within COUNTY,02198,02198
54190,99926,02198,0.000000,0.000000,1.000000,1.000000,02198,centroid intersects COUNTY,02198,02198
54191,99927,02198,0.000000,0.000000,1.000000,1.000000,02198,geometry within COUNTY,02198,02198
54192,99928,02130,0.000000,0.000000,1.000000,1.000000,NaN,single ZIP to COUNTY association,02130,02130


In [249]:
# find the zip codes with multiple county information
hm = hud_file1.ZIP.value_counts()
hm = hm[hm>1]

# identify the mapping
hud_file1.loc[(hud_file1.ZIP.isin(hm.sample(1).index)),:]

,ZIP,COUNTY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,Spatial_Overlap_COUNTY,overlay,MaxResCounty,Preferred_county
2148,05444,50011,0.503038,0.285714,0.368421,0.494786,50015,centroid intersects COUNTY,50011,50011
2149,05444,50015,0.496962,0.714286,0.631579,0.505214,50015,centroid intersects COUNTY,50011,50011


In [251]:
hud_file1.loc[hud_file1.ZIP.isin(['93001','96799','00680']),:]

,ZIP,COUNTY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,Spatial_Overlap_COUNTY,overlay,MaxResCounty,Preferred_county
73,00680,72011,0.000048,0.000000,0.0,0.000041,NaN,"geometry intersects multiple COUNTY, centroid ...",72097,72097
74,00680,72097,0.999808,0.999617,1.0,0.999796,NaN,"geometry intersects multiple COUNTY, centroid ...",72097,72097
75,00680,72023,0.000000,0.000383,0.0,0.000041,NaN,"geometry intersects multiple COUNTY, centroid ...",72097,72097
76,00680,72067,0.000144,0.000000,0.0,0.000122,NaN,"geometry intersects multiple COUNTY, centroid ...",72097,72097
50819,93001,06111,1.000000,1.000000,1.0,1.000000,NaN,"geometry intersects multiple COUNTY, centroid ...",06111,06111
52499,96799,60010,0.000000,0.000000,1.0,1.000000,NaN,"geometry intersects multiple COUNTY, centroid ...",60010,60010


In [252]:
poly_1_mapped.loc[poly_1_mapped.ZCTA5CE10.isin(['93001','96799','00680']),:]

,GEOID10,ZCTA5CE10,geometry,centroid,COUNTYFIPS,COUNTYID,overlay
1564,93001,93001,"(POLYGON ((-119.446066 34.014472, -119.443662 ...",POINT (-119.551904221347 34.16229038689207),NaN,NaN,"geometry intersects multiple COUNTY, centroid ..."
13504,96799,96799,"(POLYGON ((-169.518651 -14.221089, -169.518615...",POINT (-170.3718876669044 -14.21993791676005),NaN,NaN,"geometry intersects multiple COUNTY, centroid ..."
26399,00680,00680,"(POLYGON ((-67.15141899999999 18.177469, -67.1...",POINT (-67.41949113543808 18.15515534970313),NaN,NaN,"geometry intersects multiple COUNTY, centroid ..."


In [ ]:
# create directory if not exist
if not os.path.exists(os.path.abspath(os.path.dirname(outshapefilepath))):
    os.mkdir(os.path.abspath(os.path.dirname(outshapefilepath)))

# export to shapefiles (replace existing shapefile)
hud_file1.drop('centroid', axis=1).to_file(outshapefilepath)

# export to dataframe
hud_file1.drop(['geometry','centroid'], axis=1)\
    .to_csv(outshapefilepath.replace('.shp','.csv'), header=True, index=False)

In [265]:
hud_file1.to_csv('1-1_ZIP_COUNTY_HUD_file.csv', header=True, index=False)